In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import torchvision.models as models

resnet18 = models.resnet18()

#Categories
classes=['agriculture','artisinal_mine','bare_ground','blooming','blow_down','clear',
'cloudy','conventional_mine','cultivation','habitation','haze',
'partly_cloudy','primary','road','selective_logging','slash_burn','water']

#Classess and functions to modify layers of imported Resnet18 model
class Flatten(nn.Module):
    "Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor"
    def __init__(self, full:bool=False):
        super().__init__()
        self.full = full

    def forward(self, x):
        return x.view(-1) if self.full else x.view(x.size(0), -1)

class AdaptiveConcatPool2d(nn.Module):
	def __init__(self, sz=None):
		super().__init__()
		sz = sz or (1)
		self.ap = nn.AdaptiveAvgPool2d(sz)
		self.mp = nn.AdaptiveMaxPool2d(sz)

	def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)
    
def myhead(nf, nc):
    return \
    nn.Sequential(        # the dropout is needed otherwise you cannot load the weights
            AdaptiveConcatPool2d(),
            Flatten(full=False),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=0.25),
            nn.Linear(1024, 512, bias = False),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Dropout(p=0.5),
            nn.Linear(512, nc,bias = False),
        )

#Modifying layers of imported Resnet18 architecture
my_model=models.resnet18() 
modules=list(my_model.children())
modules.pop(-1) 
modules.pop(-1) 
temp=nn.Sequential(nn.Sequential(*modules))
tempchildren=list(temp.children()) 
tempchildren.append(myhead(512,17))
my_r18=nn.Sequential(*tempchildren)

#Preparing model for inference 
model=my_r18
weighties = torch.load('/Users/jib/Documents/models/fastai-rn18.pth',map_location=torch.device('cpu'))
model.load_state_dict(weighties['state_dict'])
model.eval()


## Inference with single image


from PIL import Image
#Change the path to where your test images are located!
imgplanet = Image.open("/Users/jib/Downloads/test-jpg/test_40478.jpg").convert('RGB')
imgplanet

#Preprocessing image
preprocess = transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )])

#Converting image to tensor
imgplanet_preprocessed = preprocess(imgplanet)
imgplanet_tensor = torch.unsqueeze(imgplanet_preprocessed, 0)
tensor_list= model.forward(imgplanet_tensor)

#Prediction
tensor_list.tolist()
tensor_list.tolist()[0]
tensor_tolist = tensor_list.tolist()[0]
indx = [i for i in range(len(tensor_tolist)) if tensor_tolist[i]>0]
print([classes[i] for i in indx ])

['agriculture', 'cultivation', 'partly_cloudy', 'primary']


In [11]:
imgplanet.show()